In [2]:
import sys
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [3]:
year = 2018
# 각 년도 및 기록 웹페이지를 딕셔너리 형태로 저장
# 타격확장 및 투수확장의 경우 규정타석/규정이닝이 아닌 전체타석/전체이닝으로 설정
record_site = {'타격기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'타격확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=WRCPLUS&o2=WAR_ALL&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP&o2=WAR&de=0&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수구속':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=14&o1=FVval&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
}

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\admin\.wdm\drivers\chromedriver\win32\105.0.5195.52]


In [5]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['타격기본'])
hit_basic_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 hit_basic_df_list에 추가
    hit_basic_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [6]:
hit_basic_df = pd.concat(hit_basic_df_list)
hit_basic_df

순    이름      팀     정렬    G   타석   타수   득점   안타  2타  ... 희타 희비     비율  \
      순    이름      팀   WAR*    G   타석   타수   득점   안타  2타  ... 희타 희비     타율   
0     1   김재환  18두LF   6.94  139  602  527  104  176  36  ...  0  7   .334   
1     2   박병호  18넥1B   6.62  113  488  400   88  138  20  ...  0  3   .345   
2     3   양의지   18두C   6.42  133  503  439   84  157  29  ...  0  6   .358   
3     4   전준우  18롯LF   5.67  144  614  556  118  190  36  ...  1  2   .342   
4     5   로하스  18KCF   5.66  144  645  564  114  172  30  ...  1  2   .305   
..  ...   ...    ...    ...  ...  ...  ...  ...  ...  ..  ... .. ..    ...   
31  328  파레디스  18두RF  -0.73   21   71   65    7    9   2  ...  0  1   .138   
32  329   이상호  18N2B  -0.73  114  204  187   32   47   2  ...  7  0   .251   
33  330   박기혁  18KSS  -0.80   98  196  173   17   37   6  ...  5  3   .214   
0   331   김용의  18L1B  -0.85  114  185  159   28   37   2  ...  7  3  0.233   
1   332   나균안   18롯C  -1.71  106  203  177   15   22   3  ...  9  0  0.124   

                                        WAR*    WPA  
       출루     장타    OPS   wOBA   wRC+   WAR*    WPA  
0    .405   .657  1.062   .438  168.8   6.94   5.80  
1    .457   .718  1.175   .478  190.9   6.62   6.44  
2    .427   .585  1.013   .431  160.5   6.42   3.91  
3    .400   .592   .991   .421  146.3   5.67   5.45  
4    .388   .590   .979   .410  138.5   5.66   5.41  
..    ...    ...    ...    ...    ...    ...    ...  
31   .197   .246   .443   .191   -2.4  -0.73  -0.65  
32   .289   .262   .551   .251   37.3  -0.73  -1.23  
33   .272   .266   .538   .244   28.9  -0.80  -1.86  
0   0.298  0.245  0.543  0.248   33.0  -0.85  -0.84  
1   0.201  0.175  0.376  0.175  -15.1  -1.71  -1.85  

[376 rows x 31 columns]

In [7]:
driver.get(record_site['타격확장'])
hit_expand_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 hit_expand_df_list에 추가
    hit_expand_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [8]:
hit_expand_df = pd.concat(hit_expand_df_list)
hit_expand_df

순   이름      팀   생산력+   타석    HR%   BB%    K%  BB/K  IsoP  ...   PSN  \
      순   이름      팀   wRC+   타석    HR%   BB%    K%  BB/K  IsoP  ...   PSN   
0     1  백상원  18삼2B  219.5    8   0.00  25.0   0.0   NaN  .000  ...   NaN   
1     2  유재신  18KLF  208.2   40   2.50  17.5  25.0  0.70  .242  ...  1.60   
2     3  박병호  18넥1B  190.9  488   8.81  13.9  23.4  0.60  .373  ...  0.00   
3     4  김종덕   18넥C  174.6    2   0.00   0.0   0.0   NaN  .000  ...   NaN   
4     5   샌즈  18넥RF  171.5   93  12.90   6.5  29.0  0.22  .454  ...  0.00   
..  ...  ...    ...    ...  ...    ...   ...   ...   ...   ...  ...   ...   
31  NaN  이민우   18KP    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
32  NaN  임창용   18KP    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
33  NaN  하준영   18KP    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
0   NaN  조상우   18넥P    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   
1   NaN  유원상   18NP    NaN    0    NaN   NaN   NaN   NaN   NaN  ...   NaN   

   타격 생산력                     타격 생산력 (파크팩터 조정)                             
     wOBA    wRC wRC/27  wRAA             wOBA    wRC wRC/27  wRAA   wRC+  
0    .526    2.4  22.02   1.3             .529    2.5  22.20   1.3  219.5  
1    .512   11.7  16.62   6.1             .512   11.7  16.62   6.1  208.2  
2    .478  127.4  12.50  58.8             .486  130.9  12.85  62.3  190.9  
3    .460    0.5  13.17   0.2             .461    0.5  13.24   0.2  174.6  
4    .445   21.4   9.17   8.3             .457   22.4   9.60   9.3  171.5  
..    ...    ...    ...   ...              ...    ...    ...   ...    ...  
31    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
32    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
33    NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
0     NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  
1     NaN    NaN    NaN   NaN              NaN    NaN    NaN   NaN    NaN  

[376 rows x 23 columns]

In [52]:
# columns의 level 0을 없애고 정렬을 위해 4번째 위치에 자리잡은 열 제거하는 함수 생성
def table_change(df):
    df = df.droplevel(0, axis=1)
    #columns = df.columns.tolist()
    # pop() 메서드로 제거하면 기본에서는 WAR*, 확장에서는 wRC+가 중복으로 발생하는데.. 왜지?
    # del 제거도 마찬가지
    #del columns[3]
    #df = df.loc[:, columns]
    return df

In [10]:
# DataFrame에 있는 불필요한 행 제거하는 함수 생성
def remove_records(df):
    df = df.query("이름 != '이름'")
    return df

In [11]:
# 기본적으로 크롤링 데이터는 str 형식이기에 숫자로 바꿔줄 필요 -> pd.to_numeric 활용할 함수 작성
# 열 정리를 다 하고 새로운 객체로 할당한 이후 적용 가능
def numeric(df):
    for col in df.columns.tolist():
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

In [54]:
hit_basic_cols = ['순', '이름', '팀', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런',
       '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비',
       '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA']
hit_basic_df2 = (hit_basic_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, hit_basic_cols]
)
# 중복되어 나오는 문제를 해결하기 위해 열 drop 후 새로운 Series로 추가
hit_basic_war = hit_basic_df2.loc[:, 'WAR*'].iloc[:, 0]
hit_basic_df3 = (hit_basic_df2
.drop(['WAR*', '순'], axis=1)
.assign(**{'WAR*':hit_basic_war})
)
hit_basic_df3

,이름,팀,G,타석,타수,득점,안타,2타,3타,홈런,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WPA,WAR*
0,김재환,18두LF,139,602,527,104,176,36,1,44,...,0,7,.334,.405,.657,1.062,.438,168.8,5.80,6.94
1,박병호,18넥1B,113,488,400,88,138,20,0,43,...,0,3,.345,.457,.718,1.175,.478,190.9,6.44,6.62
2,양의지,18두C,133,503,439,84,157,29,1,23,...,0,6,.358,.427,.585,1.013,.431,160.5,3.91,6.42
3,전준우,18롯LF,144,614,556,118,190,36,2,33,...,1,2,.342,.400,.592,.991,.421,146.3,5.45,5.67
4,로하스,18KCF,144,645,564,114,172,30,1,43,...,1,2,.305,.388,.590,.979,.410,138.5,5.41,5.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,파레디스,18두RF,21,71,65,7,9,2,1,1,...,0,1,.138,.197,.246,.443,.191,-2.4,-0.65,-0.73
32,이상호,18N2B,114,204,187,32,47,2,0,0,...,7,0,.251,.289,.262,.551,.251,37.3,-1.23,-0.73
33,박기혁,18KSS,98,196,173,17,37,6,0,1,...,5,3,.214,.272,.266,.538,.244,28.9,-1.86,-0.80
0,김용의,18L1B,114,185,159,28,37,2,0,0,...,7,3,0.233,0.298,0.245,0.543,0.248,33.0,-0.84,-0.85


In [55]:
hit_expand_cols = ['순', '이름', '팀', '타석', 'HR%', 'BB%', 'K%', 'BB/K', 'IsoP', 'IsoD',
       'BABIP', 'Spd', 'PSN', 'wRC+']
hit_expand_df2 = (hit_expand_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, hit_expand_cols]
)
# 중복되어 나오는 문제를 해결하기 위해 열 drop 후 새로운 Series로 추가
hit_expand_wrc = hit_expand_df2.loc[:, 'wRC+'].iloc[:, 0]
hit_expand_df3 = (hit_expand_df2
.drop(['wRC+', '순'], axis=1)
.assign(**{'wRC+':hit_expand_wrc})
)
hit_expand_df3

,이름,팀,타석,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN,wRC+
0,백상원,18삼2B,8,0.00,25.0,0.0,NaN,.000,.125,.500,2.4,NaN,219.5
1,유재신,18KLF,40,2.50,17.5,25.0,0.70,.242,.101,.591,8.0,1.60,208.2
2,박병호,18넥1B,488,8.81,13.9,23.4,0.60,.373,.112,.386,1.7,0.00,190.9
3,김종덕,18넥C,2,0.00,0.0,0.0,NaN,.000,.000,.500,1.9,NaN,174.6
4,샌즈,18넥RF,93,12.90,6.5,29.0,0.22,.454,.041,.313,3.8,0.00,171.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,이민우,18KP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,임창용,18KP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,하준영,18KP,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,조상우,18넥P,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# 타격-기본과 타격-확장을 합쳐주기 위해 merge 활용
hit_df = (hit_basic_df3
.merge(hit_expand_df3, on=['이름', '팀', '타석', 'wRC+'], how='inner')
.pipe(numeric)
)
hit_df.columns

Index(['이름', '팀', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
       '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타',
       'OPS', 'wOBA', 'wRC+', 'WPA', 'WAR*', 'HR%', 'BB%', 'K%', 'BB/K',
       'IsoP', 'IsoD', 'BABIP', 'Spd', 'PSN'],
      dtype='object')

In [57]:
(hit_df
.loc[:, '팀']
.str.slice(0, 2)
)

0      18
1      18
2      18
3      18
4      18
       ..
328    18
329    18
330    18
331    18
332    18
Name: 팀, Length: 333, dtype: object

In [58]:
hit_columns = ['이름', '연도', '소속', '포지션', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
       '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타',
       'OPS', 'wOBA', 'wRC+', 'WPA', 'WAR*', 'HR%', 'BB%', 'K%', 'BB/K',
       'IsoP', 'IsoD', 'BABIP', 'Spd', 'PSN']

In [59]:
(hit_df
.assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
소속=lambda df: df['팀'].str.extract('(L|키|K|S|N|한|삼|롯|두|넥)')
.replace({'L':'LG', '키':'넥센/키움', 'S':'SK/SSG', 'N':'NC', '한':'한화', '삼':'삼성', '롯':'롯데', '두':'두산', '넥':'넥센/키움'}),
포지션=lambda df: df['팀'].str.extract('(LF|CF|RF|1B|2B|3B|SS|C|DH|P)'))
.loc[:, hit_columns]
)

,이름,연도,소속,포지션,G,타석,타수,득점,안타,2타,...,WAR*,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,김재환,2018,두산,LF,139,602,527,104,176,36,...,6.94,7.31,9.8,22.3,0.44,0.323,0.071,0.371,3.5,3.83
1,박병호,2018,넥센/키움,1B,113,488,400,88,138,20,...,6.62,8.81,13.9,23.4,0.60,0.373,0.112,0.386,1.7,0.00
2,양의지,2018,두산,C,133,503,439,84,157,29,...,6.42,4.57,9.0,8.0,1.13,0.228,0.070,0.351,3.9,9.52
3,전준우,2018,롯데,LF,144,614,556,118,190,36,...,5.67,5.38,7.8,13.4,0.59,0.250,0.058,0.354,4.1,11.55
4,로하스,2018,K,CF,144,645,564,114,172,30,...,5.66,6.67,11.0,22.0,0.50,0.286,0.083,0.339,4.2,25.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,안익훈,2018,LG,CF,62,135,123,13,27,4,...,-0.70,0.00,5.9,8.9,0.67,0.033,0.051,0.241,2.6,NaN
329,이종욱,2018,NC,LF,43,128,122,12,28,7,...,-0.71,0.78,3.1,12.5,0.25,0.082,0.022,0.255,3.4,1.00
330,파레디스,2018,두산,RF,21,71,65,7,9,2,...,-0.73,1.41,5.6,23.9,0.24,0.108,0.059,0.167,6.8,1.00
331,이상호,2018,NC,2B,114,204,187,32,47,2,...,-0.73,0.00,3.9,7.4,0.53,0.011,0.038,0.273,5.0,0.00


In [18]:
kt_site = {'타격':f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys={year}&ye={year}&se=0&te=kt&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=0&si=&cn=',
'투수':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=kt&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='}

In [19]:
kt_df_list = []
driver.get(kt_site['타격'])
while True:
    page = driver.page_source
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 kt_df_list에 추가
    kt_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [20]:
kt_team_member = (pd.concat(kt_df_list)
.pipe(table_change)
['이름']
.tolist()
)
kt_team_member

['로하스',
 '유한준',
 '황재균',
 '박경수',
 '강백호',
 '장성우',
 '윤석민',
 '이진영',
 '이대형',
 '김재윤',
 '이름',
 '이름',
 '홍성용',
 '신병률',
 '엄상백',
 '고창성',
 '이종혁',
 '김사율',
 '홍현빈',
 '이준수',
 '고명성',
 '정현',
 '이름',
 '이름',
 '심우준',
 '오태곤',
 '김영환',
 '배정대',
 '김지열',
 '남태혁',
 '이해창',
 '김동욱',
 '오정복',
 '이창진',
 '전민수',
 '김진곤',
 '하준호',
 '오준혁',
 '송민섭',
 '박기혁']

In [60]:
(hit_df
.assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
소속=lambda df: df['팀'].str.extract('(L|키|K|S|N|한|삼|롯|두|넥)')
.replace({'L':'LG', '키':'넥센/키움', 'K':'KIA', 'S':'SK/SSG', 'N':'NC', '한':'한화', '삼':'삼성', '롯':'롯데', '두':'두산', '넥':'넥센/키움'}),
포지션=lambda df: df['팀'].str.extract('(LF|CF|RF|1B|2B|3B|SS|C|DH|P)'))
# in 대신 .isin()메서드를 써야하는 걸 한참을 삽질..
# in은 Series 객체 전체와 비교 진행 → The truth value of a Series is ambiguous 에러 발생
.assign(소속=lambda df: np.where(df['이름'].isin(kt_team_member), 'KT', df['소속']))
.loc[:, hit_columns]
)

,이름,연도,소속,포지션,G,타석,타수,득점,안타,2타,...,WAR*,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,김재환,2018,두산,LF,139,602,527,104,176,36,...,6.94,7.31,9.8,22.3,0.44,0.323,0.071,0.371,3.5,3.83
1,박병호,2018,넥센/키움,1B,113,488,400,88,138,20,...,6.62,8.81,13.9,23.4,0.60,0.373,0.112,0.386,1.7,0.00
2,양의지,2018,두산,C,133,503,439,84,157,29,...,6.42,4.57,9.0,8.0,1.13,0.228,0.070,0.351,3.9,9.52
3,전준우,2018,롯데,LF,144,614,556,118,190,36,...,5.67,5.38,7.8,13.4,0.59,0.250,0.058,0.354,4.1,11.55
4,로하스,2018,KT,CF,144,645,564,114,172,30,...,5.66,6.67,11.0,22.0,0.50,0.286,0.083,0.339,4.2,25.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,안익훈,2018,LG,CF,62,135,123,13,27,4,...,-0.70,0.00,5.9,8.9,0.67,0.033,0.051,0.241,2.6,NaN
329,이종욱,2018,NC,LF,43,128,122,12,28,7,...,-0.71,0.78,3.1,12.5,0.25,0.082,0.022,0.255,3.4,1.00
330,파레디스,2018,두산,RF,21,71,65,7,9,2,...,-0.73,1.41,5.6,23.9,0.24,0.108,0.059,0.167,6.8,1.00
331,이상호,2018,NC,2B,114,204,187,32,47,2,...,-0.73,0.00,3.9,7.4,0.53,0.011,0.038,0.273,5.0,0.00


In [22]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['투수기본'])
pitch_basic_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 pitch_basic_df_list에 추가
    pitch_basic_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [23]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['투수확장'])
pitch_expand_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 pitch_expand_df_list에 추가
    pitch_expand_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [24]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['투수구속'])
pitch_speed_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 pitch_speed_df_list에 추가
    pitch_speed_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [61]:
pitch_basic_cols = ['순', '이름', '팀', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '이닝', '실점',
       '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투',
       'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
pitch_basic_df = pd.concat(pitch_basic_df_list)
pitch_basic_df2 = (pitch_basic_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, pitch_basic_cols]
)
pitch_basic_war = pitch_basic_df2.loc[:, 'WAR'].iloc[:, 0]
pitch_basic_df3 = (pitch_basic_df2
.drop(['WAR', '순'], axis=1)
.assign(**{'WAR':pitch_basic_war})
)
pitch_basic_df3

,이름,팀,출장,완투,완봉,선발,승,패,세,홀드,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA,WAR
0,린드블럼,18두,26,0,0,26,15,4,0,0,...,157,0,10,2.88,4.02,1.07,175.4,125.2,2.06,6.83
1,윌슨,18L,26,1,1,26,9,4,0,0,...,149,1,10,3.07,3.63,1.14,164.3,139.4,2.72,6.33
2,브리검,18넥,31,2,1,30,11,7,0,1,...,175,0,4,3.84,4.36,1.20,134.1,117.4,0.20,5.80
3,김광현,18S,25,0,0,25,11,8,0,0,...,130,0,3,2.98,4.16,1.14,176.5,127.4,1.91,5.28
4,양현종,18K,29,3,0,29,13,11,0,0,...,152,0,5,4.15,4.38,1.31,123.7,117.9,-1.06,5.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,박세진,18K,8,0,0,8,1,5,0,0,...,24,0,3,10.68,8.42,2.18,49.6,61.7,-1.27,-0.94
18,주권,18K,46,0,0,10,3,9,0,4,...,54,0,4,8.39,6.09,1.65,63.2,85.1,-2.79,-1.03
19,김대현,18L,25,0,0,21,2,10,0,0,...,64,2,7,7.54,6.10,1.84,66.9,82.2,-3.86,-1.18
20,박세웅,18롯,14,0,0,12,1,5,0,0,...,40,0,3,9.92,6.87,2.29,52.6,76.1,-2.21,-1.20


In [62]:
pitch_expand_cols = ['순', '이름', '팀', '출장', '이닝', 'ERA', 'FIP', 'K/9', 'BB/9', 'K/BB',
       'HR/9', 'K%', 'BB%', 'K-BB%', 'PFR', 'BABIP', 'LOB%', '타율', '출루율',
       '장타율', 'OPS', 'WHIP', 'WHIP+', '투구', 'IP/G', 'P/G', 'P/IP', 'P/PA',
       'CYP']
pitch_expand_df = pd.concat(pitch_expand_df_list)
pitch_expand_df2 = (pitch_expand_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, pitch_expand_cols]
)
pitch_expand_fip = pitch_expand_df2.loc[:, 'FIP'].iloc[:, 0]
pitch_expand_df3 = (pitch_expand_df2
.drop(['FIP', '순'], axis=1)
.assign(**{'FIP':pitch_expand_fip})
)
pitch_expand_df3

,이름,팀,출장,이닝,ERA,K/9,BB/9,K/BB,HR/9,K%,...,OPS,WHIP,WHIP+,투구,IP/G,P/G,P/IP,P/PA,CYP,FIP
0,장지훈,18삼,1,1.0,0.00,9.00,0.00,NaN,0.00,25.0,...,.500,1.00,1.00,16,1.0,16.0,16.0,4.00,0.6,1.83
1,배우열,18K,5,5.2,1.59,14.29,4.77,3.00,0.00,32.1,...,.677,1.77,1.77,109,1.1,21.8,19.2,3.89,2.7,2.25
2,손승락,18롯,57,57.2,3.90,9.36,3.12,3.00,0.16,24.0,...,.604,1.27,1.28,994,1.0,17.4,17.2,3.98,12.8,2.86
3,이상화,18K,11,10.1,2.61,12.19,4.36,2.80,0.00,32.6,...,.529,1.07,1.16,184,0.9,16.7,17.8,4.28,3.6,2.87
4,최충연,18삼,70,85.0,3.60,10.69,2.75,3.89,0.64,29.4,...,.650,1.17,1.19,1384,1.2,19.8,16.3,4.02,20.6,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,김민규,18두,1,0.1,54.00,0.00,54.00,0.00,0.00,0.0,...,1.250,9.00,9.00,17,0.3,17.0,51.0,4.25,-1.8,21.83
18,이우찬,18L,3,0.2,54.00,0.00,40.50,0.00,0.00,0.0,...,1.778,9.00,10.50,32,0.2,10.7,48.0,3.56,-3.7,21.83
19,장민익,18두,3,2.0,27.00,9.00,9.00,1.00,13.50,14.3,...,1.844,3.50,4.00,61,0.7,20.3,30.5,4.36,-4.8,25.83
20,김기태,18삼,1,1.2,27.00,10.80,0.00,NaN,16.20,20.0,...,1.944,2.40,3.00,42,1.7,42.0,25.2,4.20,-4.0,26.63


In [85]:
pitch_speed_cols = ['순', '이름', '팀', '출장', '이닝', '직구', '슬라', '커브', '첸졉', '스플', '싱커',
       '너클', '기타']
pitch_speed_df = pd.concat(pitch_speed_df_list)
pitch_speed_df2 = (pitch_speed_df
.pipe(table_change)
.pipe(remove_records)
.loc[:, pitch_speed_cols]
)
pitch_speed_df2.columns = ['순', '이름', '팀', '출장', '이닝', '직구구속', '직구구속2', '직구구사', '슬라구속', '슬라구사', '커브구속', '커브구사',
       '첸졉구속', '첸졉구사', '스플구속', '스플구사', '싱커구속', '싱커구사', '너클구속', '너클구사', '기타구속', '기타구사']
pitch_speed_df2
pitch_speed_df3 = (pitch_speed_df2
.drop('직구구속2', axis=1)
.loc[:, ['순', '이름', '팀', '출장', '이닝', '직구구속', '슬라구속', '커브구속', '첸졉구속', '스플구속', '싱커구속', '너클구속', '기타구속', 
       '직구구사', '슬라구사', '커브구사', '첸졉구사', '스플구사', '싱커구사', '너클구사', '기타구사']]
)
pitch_speed_df3

,순,이름,팀,출장,이닝,직구구속,슬라구속,커브구속,첸졉구속,스플구속,...,너클구속,기타구속,직구구사,슬라구사,커브구사,첸졉구사,스플구사,싱커구사,너클구사,기타구사
0,1,조상우,18넥,18,19.0,150.9,136.7,121.0,NaN,140.0,...,NaN,NaN,68.3,29.5,1.9,0.0,0.3,0.0,0.0,0.0
1,2,산체스,18S,29,145.1,150.7,143.6,128.9,140.0,136.6,...,NaN,NaN,50.9,21.5,11.3,12.8,2.9,0.6,0.0,0.0
2,3,강지광,18S,4,3.0,150.2,139.5,126.0,NaN,135.0,...,NaN,NaN,78.3,5.8,2.9,0.0,13.0,0.0,0.0,0.0
3,4,소사,18L,27,181.1,149.5,135.8,126.4,NaN,132.6,...,NaN,NaN,48.8,17.0,1.7,0.0,30.3,2.2,0.0,0.0
4,5,한승혁,18K,21,88.0,149.2,136.1,116.8,132.0,135.3,...,NaN,NaN,54.7,15.2,10.3,0.1,19.7,0.1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,256,박종훈,18S,30,159.1,131.6,130.0,119.1,123.8,124.6,...,NaN,NaN,41.1,0.0,42.6,3.7,0.3,12.2,0.0,0.0
18,NaN,김대우,18삼,8,36.1,131.6,117.8,NaN,NaN,NaN,...,NaN,NaN,41.5,45.8,0.0,0.0,0.0,12.7,0.0,0.0
19,258,양현,18넥,33,32.2,130.4,125.8,116.2,122.2,NaN,...,NaN,NaN,12.6,1.0,34.6,2.3,0.0,49.6,0.0,0.0
20,259,유희관,18두,29,141.0,129.6,121.1,101.6,116.7,116.0,...,NaN,NaN,42.5,12.7,10.8,0.1,3.9,29.9,0.0,0.0
